## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [5]:
print('Instrument:', instrument)

Instrument: EUR_GBP


### The Configs for Run:

In [6]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [7]:
Load_10K_Records=True
Load_20K_Records=True

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [8]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [9]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [10]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [11]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
       response=second_response 
else:
    response=first_response['candles']

In [12]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [13]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data.shape

(19341, 12)

In [15]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2009-11-13,10:00:00,10,317,4,Friday,2009-11-13T10:00:00.000000000Z,5545,0.89202,0.89330,0.89098,0.89164
1,2009-11-13,14:00:00,14,317,4,Friday,2009-11-13T14:00:00.000000000Z,7988,0.89166,0.89461,0.89069,0.89378
2,2009-11-13,18:00:00,18,317,4,Friday,2009-11-13T18:00:00.000000000Z,2920,0.89380,0.89485,0.89274,0.89373
3,2009-11-13,22:00:00,22,317,4,Friday,2009-11-13T22:00:00.000000000Z,2,0.89378,0.89378,0.89378,0.89378
4,2009-11-15,10:00:00,10,319,6,Sunday,2009-11-15T10:00:00.000000000Z,1,0.89378,0.89378,0.89378,0.89378


In [16]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
19336,2022-01-28,10:00:00,10,28,4,Friday,2022-01-28T10:00:00.000000000Z,4482,0.83190,0.83271,0.83120,0.83182
19337,2022-01-28,14:00:00,14,28,4,Friday,2022-01-28T14:00:00.000000000Z,6339,0.83178,0.83307,0.83055,0.83290
19338,2022-01-28,18:00:00,18,28,4,Friday,2022-01-28T18:00:00.000000000Z,1912,0.83292,0.83342,0.83171,0.83171
19339,2022-01-30,22:00:00,22,30,6,Sunday,2022-01-30T22:00:00.000000000Z,1461,0.83177,0.83235,0.83136,0.83186
19340,2022-01-31,02:00:00,2,31,0,Monday,2022-01-31T02:00:00.000000000Z,55,0.83184,0.83199,0.83176,0.83197


## Simple Moving Average (SMA)

In [17]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [18]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [19]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [20]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
19336,2022-01-28,10:00:00,10,28,4,Friday,2022-01-28T10:00:00.000000000Z,4482,0.83190,0.83271,0.83120,0.83182,0.8321,0.8331,0.8346,-0.00028,-0.00128,-0.00278
19337,2022-01-28,14:00:00,14,28,4,Friday,2022-01-28T14:00:00.000000000Z,6339,0.83178,0.83307,0.83055,0.83290,0.8321,0.8329,0.8344,0.00080,0.00000,-0.00150
19338,2022-01-28,18:00:00,18,28,4,Friday,2022-01-28T18:00:00.000000000Z,1912,0.83292,0.83342,0.83171,0.83171,0.8320,0.8325,0.8341,-0.00029,-0.00079,-0.00239
19339,2022-01-30,22:00:00,22,30,6,Sunday,2022-01-30T22:00:00.000000000Z,1461,0.83177,0.83235,0.83136,0.83186,0.8320,0.8323,0.8339,-0.00014,-0.00044,-0.00204
19340,2022-01-31,02:00:00,2,31,0,Monday,2022-01-31T02:00:00.000000000Z,55,0.83184,0.83199,0.83176,0.83197,0.8321,0.8322,0.8337,-0.00013,-0.00023,-0.00173


## Price Range

In [21]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [22]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [23]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
19317,2022-01-28,10:00:00,10,28,4,Friday,2022-01-28T10:00:00.000000000Z,4482,0.83190,0.83271,...,0.00008,0.00151,0.00089,-0.00062,0,0.00008,0.00089,0.00070,-0.00026,0.00073
19318,2022-01-28,14:00:00,14,28,4,Friday,2022-01-28T14:00:00.000000000Z,6339,0.83178,0.83307,...,-0.00112,0.00252,0.00017,-0.00235,1,-0.00112,0.00017,0.00123,0.00008,-0.00026
19319,2022-01-28,18:00:00,18,28,4,Friday,2022-01-28T18:00:00.000000000Z,1912,0.83292,0.83342,...,0.00121,0.00171,0.00171,0.00000,0,0.00121,0.00171,0.00121,-0.00112,0.00008
19320,2022-01-30,22:00:00,22,30,6,Sunday,2022-01-30T22:00:00.000000000Z,1461,0.83177,0.83235,...,-0.00009,0.00099,0.00049,-0.00050,1,-0.00009,0.00049,0.00041,0.00121,-0.00112
19321,2022-01-31,02:00:00,2,31,0,Monday,2022-01-31T02:00:00.000000000Z,55,0.83184,0.83199,...,-0.00013,0.00023,0.00002,-0.00021,1,-0.00013,0.00002,0.00008,-0.00009,0.00121


In [24]:
data.shape

(19322, 30)